In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras import models, layers
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
df = pd.read_csv('coords.csv')


In [ ]:
df.head()


In [ ]:
df[df['class']=='Horse Stance']


In [ ]:
# Encode class labels to numerical values
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

In [ ]:
# Separate features (X) and target (y)
X = df.drop('class', axis=1)
y = df['class']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234, stratify=y)

In [ ]:
# Define the model
def create_model():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the model
model = create_model()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
model.summary()


In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')

# Confusion Matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
class_report = classification_report(y_test, y_pred_classes)
print('Classification Report:')
print(class_report)

In [ ]:
# K-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)
cv_scores = []

for train_index, test_index in kf.split(X, y):
    X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
    y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

    model_cv = create_model()
    model_cv.fit(X_train_cv, y_train_cv, epochs=10, batch_size=32, validation_split=0.2, verbose=0)
    
    _, accuracy_cv = model_cv.evaluate(X_test_cv, y_test_cv)
    cv_scores.append(accuracy_cv)

print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))

In [ ]:
# Save the model in the TensorFlow SavedModel format
model.save('saved_model')

# Convert the model to TensorFlow Lite format (optional)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import tensorflow as tf

# Load the SavedModel
saved_model_path = './saved_model'  # Adjust the path accordingly
model = tf.saved_model.load(saved_model_path)

# Load the TensorFlow Lite model
tflite_model_path = './model.tflite'  # Adjust the path accordingly
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize BlazePose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Start capturing video from the camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1200)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with BlazePose
    results = pose.process(frame_rgb)

    # Recolor image back to BGR for rendering
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    # Detect Taijiquan Stances
    if results.pose_landmarks:
        # Extract Pose landmarks
        pose_landmarks = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose_landmarks]).flatten())

        # Make Detections
        X = pd.DataFrame([pose_row])

        # Convert X to numpy array
        input_data = X.to_numpy().astype(np.float32)

        # Set the input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Run inference
        interpreter.invoke()

        # Get the output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])

        # Get the predicted class and probabilities
        body_language_class = np.argmax(output_data)
        body_language_prob = output_data[0]

        print(body_language_class, body_language_prob)

        # Display Probability
        cv2.putText(frame, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, str(round(body_language_prob[body_language_class], 2)),
                    (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Display detected class
        cv2.putText(frame, f'CLASS: {body_language_class}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

    cv2.imshow('Pose Detection with ML', frame)

    # Check for exit key
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import pairwise_distances

# Load your model and data
saved_model_path = './saved_model'  # Adjust the path accordingly
model = tf.saved_model.load(saved_model_path)

tflite_model_path = './model.tflite'  # Adjust the path accordingly
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Define the columns to exclude (assuming you want to exclude z and v coordinates)
columns_to_exclude = [f'z{i}' for i in range(1, 34)] + [f'v{i}' for i in range(1, 34)]

# Select the columns for x and y coordinates from X_train excluding the specified columns
selected_columns = [col for col in X_train.columns if col not in columns_to_exclude]
ground_truth_keypoints = X_train[selected_columns].values

# Use the model to predict keypoints on the training data
predicted_keypoints = []

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

for i in range(len(X_train)):
    # Extract Pose landmarks from training data
    pose_row = X_train.iloc[i].values  # Assuming your data is in the correct format

    # Make Detections
    X = pd.DataFrame([pose_row])

    # Convert X to numpy array
    input_data = X.to_numpy().astype(np.float32)

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run inference
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Get the predicted class and probabilities
    body_language_class = np.argmax(output_data)

    # Append the predicted keypoints to the list
    predicted_keypoints.append(np.array([[landmark[0], landmark[1]] for landmark in pose_row]))

# Convert the list of predicted keypoints to a NumPy array
predicted_keypoints = np.array(predicted_keypoints).reshape(-1, 33, 2)  # Reshape to 3D

# Calculate PCK
distances = pairwise_distances(predicted_keypoints.reshape(-1, 2), ground_truth_keypoints.reshape(-1, 2))
num_correct_keypoints = np.sum(distances < distance_threshold)

# Calculate and display PCK
pck = (num_correct_keypoints / (len(ground_truth_keypoints) * len(X_train))) * 100
print(f'PCK on training data: {pck:.2f}%')